In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import os

In [ ]:
class ImageNetMiniDataset(Dataset):
    def __init__(self, txt_file, root_dir, transform=None):
        """
        txt_file: .txt 檔案路徑（例如 train.txt）
        root_dir: 圖像資料夾根目錄（例如 .）
        transform: 圖像預處理變換
        """
        self.root_dir = root_dir
        self.transform = transform
        # 讀取 .txt 檔案，存儲圖像路徑和標籤
        self.data = []
        with open(txt_file, 'r') as f:
            for line in f:
                image_path, label = line.strip().split()
                self.data.append((image_path, int(label)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        # 拼接完整圖像路徑
        image_path = os.path.join(self.root_dir, image_path)
        # 讀取圖像
        image = Image.open(image_path).convert('RGB')  # 轉為 RGB 格式
        # 應用變換（如果有）
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定義圖像預處理（ResNet 標準預處理）
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 調整大小為 224x224
    transforms.ToTensor(),  # 轉為 Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet 均值
                        std=[0.229, 0.224, 0.225])  # ImageNet 標準差
])

# 加載數據集
root_dir = "."  # 根目錄（根據你的實際路徑調整）
train_dataset = ImageNetMiniDataset(txt_file="dataset/train.txt", root_dir=root_dir, transform=transform)
val_dataset = ImageNetMiniDataset(txt_file="dataset/val.txt", root_dir=root_dir, transform=transform)
test_dataset = ImageNetMiniDataset(txt_file="dataset/test.txt", root_dir=root_dir, transform=transform)

# 創建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# 定義 ResNet34 對照組模型（不修改任何結構）
class ResNet34Baseline(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet34Baseline, self).__init__()
        # 直接載入 torchvision 中的 ResNet34 模型，不使用預訓練權重
        self.model = models.resnet34(pretrained=False)
        # 確認全連接層的輸出類別數（ImageNet 為 1000 類）
        # ResNet34 的全連接層預設已經適配 1000 類，因此這裡不需要修改
        # 如果 num_classes 不等於 1000，可以替換全連接層如下：
        if num_classes != 1000:
            self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# 測試模型
def test_model():
    # 創建模型實例
    model = ResNet34Baseline(num_classes=1000)
    # 模擬輸入：批次大小 4，3 通道，224x224 圖像
    x = torch.randn(4, 3, 224, 224)
    # 前向傳播
    output = model(x)
    print(f"輸出形狀：{output.shape}")  # 應為 (4, 1000)
    # 計算參數量
    total_params = sum(p.numel() for p in model.parameters())
    print(f"模型總參數量：{total_params}")

In [ ]:
def main():
    num_classes = 50

    # 初始化模型
    model = ResNet34Baseline(num_classes=num_classes).to(device)
    print(f"模型總參數量：{sum(p.numel() for p in model.parameters())}")

    # 定義損失函數和優化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # 訓練迴圈
    num_epochs = 10  # 訓練 10 個 epoch（可調整）
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # 前向傳播
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 反向傳播和優化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 計算損失和準確率
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}] 訓練損失：{epoch_loss:.4f} 訓練準確率：{epoch_acc:.2f}%")

        # 驗證階段
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_acc = 100 * val_correct / val_total
        print(f"驗證準確率：{val_acc:.2f}%")

    # 測試階段
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    test_acc = 100 * test_correct / test_total
    print(f"測試準確率：{test_acc:.2f}%")

if __name__ == "__main__":
    main()

## Mynet

In [ ]:
import torch
import torch.nn as nn

# 定義基本的殘差塊
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        # 第一個卷積層，可能改變空間尺寸（通過 stride）
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        # 第二個卷積層，保持尺寸不變
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # 殘差連接：如果輸入輸出通道數或尺寸不匹配，則用 1x1 卷積調整
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        # 殘差連接：將輸入（經過調整）與輸出相加
        out += self.shortcut(identity)
        out = self.relu(out)
        return out

# 定義簡化的 ResNet 模型
class SimpleResNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(SimpleResNet, self).__init__()
        # 初始卷積層：7x7 卷積，64 個濾波器，步幅 2
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        # 最大池化層：3x3，步幅 2
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # 殘差塊 1：輸入 64 通道，輸出 64 通道，步幅 1
        self.block1 = ResidualBlock(64, 64, stride=1)
        # 殘差塊 2：輸入 64 通道，輸出 128 通道，步幅 2（降採樣）
        self.block2 = ResidualBlock(64, 128, stride=2)

        # 全局平均池化
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # 全連接層：128 通道到分類數量
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # 初始卷積和池化
        x = self.conv1(x)  # 輸入：224x224x3，輸出：112x112x64
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)  # 輸出：56x56x64

        # 殘差塊
        x = self.block1(x)  # 輸出：56x56x64
        x = self.block2(x)  # 輸出：28x28x128

        # 全局平均池化和全連接層
        x = self.avgpool(x)  # 輸出：1x1x128
        x = torch.flatten(x, 1)  # 展平為 (batch_size, 128)
        x = self.fc(x)  # 輸出：(batch_size, num_classes)
        return x

# 測試模型
def test_model():
    # 創建模型實例
    model = SimpleResNet(num_classes=1000)
    # 模擬輸入：批次大小 4，3 通道，224x224 圖像
    x = torch.randn(4, 3, 224, 224)
    # 前向傳播
    output = model(x)
    print(f"輸出形狀：{output.shape}")  # 應為 (4, 1000)
    # 計算參數量
    total_params = sum(p.numel() for p in model.parameters())
    print(f"模型總參數量：{total_params}")

if __name__ == "__main__":
    test_model()